In [134]:
!pip install geopandas --upgrade --q
!pip install gtfs_kit --upgrade --q

In [135]:
import sys
from pathlib import Path
import json

import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib

DIR = Path('..')
sys.path.append(str(DIR))

import gtfs_kit as gk

DATA_DIR = DIR/'content/'

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [136]:
# Read feed and describe/content/NorthernKentucky_TANK.zip

path = DATA_DIR/'gtfs (4).zip'
feed = gk.read_feed(path, dist_units='km')
print(feed)

* agency --------------------
	   agency_id  ... agency_email
0  Annapolis  ...          NaN

[1 rows x 8 columns]
* calendar --------------------
	  service_id  monday  tuesday  wednesday  ...  saturday  sunday  start_date  end_date
0      35333       1        1          1  ...         0       0    20180801  20210801
1      35331       0        0          0  ...         1       0    20180801  20210801
2      35332       0        0          0  ...         0       1    20180801  20210801
3      35329       1        1          1  ...         1       0    20180801  20210801

[4 rows x 10 columns]
* calendar_dates --------------------
	  service_id      date  exception_type
0      35333  20180903               2
1      35333  20181112               2
2      35333  20181122               2
3      35333  20181225               2
4      35333  20190101               2
* fare_attributes --------------------
	None
* fare_rules --------------------
	None
* feed_info --------------------
	  feed_

In [137]:
feed.describe()

,indicator,value
0,agencies,[Annapolis Transit]
1,timezone,America/New_York
2,start_date,20180801
3,end_date,20210801
4,num_routes,10
5,num_trips,415
6,num_stops,237
7,num_shapes,18
8,sample_date,20180809
9,num_routes_active_on_sample_date,10


In [138]:
## To check the warnings in the datas
feed.validate()

,type,message,table,rows
1,error,First/last/time point arrival/departure time m...,stop_times,"[6574, 6575, 6576, 6578, 6579, 6580, 6581, 658..."
0,warning,Feed expired,calendar/calendar_dates,[]
2,warning,"Repeated pair (trip_id, departure_time)",stop_times,"[6575, 6576, 6578, 6579, 6580, 6581, 6582, 658..."
3,warning,Unrecognized column block_name,trips,[]


In [139]:
# List feed
gk.list_feed(path)

,file_name,file_size
0,agency.txt,259
1,calendar.txt,240
2,calendar_dates.txt,439
3,feed_info.txt,151
4,frequencies.txt,152
5,routes.txt,651
6,shapes.txt,403349
7,stop_times.txt,303375
8,stops.txt,18627
9,trips.txt,29244


In [140]:
## Compute the trip stats
trip_stats=feed.compute_trip_stats()
trip_stats.head().T

,32,33,34,35,36
trip_id,2660017,2660018,2660019,2660020,2660021
route_id,25492,25492,25492,25492,25492
route_short_name,Orange,Orange,Orange,Orange,Orange
route_type,3,3,3,3,3
direction_id,0,0,0,0,0
shape_id,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760
num_stops,22,22,22,22,22
start_time,05:30:00,06:15:00,07:00:00,07:45:00,08:30:00
end_time,05:51:00,06:36:00,07:21:00,08:06:00,08:51:00
start_stop_id,1025801,1025801,1025801,1025801,1025801


In [141]:
# Add shape_dist_traveled column to stop times
feed = feed.append_dist_to_stop_times()

In [142]:
feed.stop_times.head().T

,0,1,2,3,4
trip_id,2659985,2659985,2659985,2659985,2659985
arrival_time,07:15:00,NaN,NaN,NaN,07:21:00
departure_time,07:15:00,NaN,NaN,NaN,07:21:00
stop_id,1025862,1025881,1025921,1025764,1025765
stop_sequence,0,1,2,3,4
timepoint,1,1,1,1,1
shape_dist_traveled,NaN,NaN,NaN,NaN,1.98464


In [143]:
## Choose the study dates 
week=feed.get_first_week()
week

['20180806',
 '20180807',
 '20180808',
 '20180809',
 '20180810',
 '20180811',
 '20180812']

In [144]:
date=[week[0],week[4]] ##  is counted as sunday and 4-wdnesday
date

['20180806', '20180810']

In [145]:
## COmpute the feed time series
fts=feed.compute_feed_time_series(trip_stats,date,freq='60Min')

In [146]:
fts

indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2018-08-06 00:00:00,0.0,0.0,0.0,0.000000,0.000000,0.000000
2018-08-06 01:00:00,0.0,0.0,0.0,0.000000,0.000000,0.000000
2018-08-06 02:00:00,0.0,0.0,0.0,0.000000,0.000000,0.000000
2018-08-06 03:00:00,0.0,0.0,0.0,0.000000,0.000000,0.000000
2018-08-06 04:00:00,0.0,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...
2018-08-10 19:00:00,4.0,4.0,6.0,81.428631,2.866667,28.405337
2018-08-10 20:00:00,4.0,4.0,6.0,42.367025,1.600000,26.479391
2018-08-10 21:00:00,3.0,2.0,4.0,44.560005,1.666667,26.736003


In [148]:
gk.downsample(fts,freq='24H')

indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2018-08-06,250.0,250.0,250.0,2859.545748,117.783333,24.278017
2018-08-07,NaN,NaN,NaN,NaN,NaN,NaN
2018-08-08,NaN,NaN,NaN,NaN,NaN,NaN
2018-08-09,NaN,NaN,NaN,NaN,NaN,NaN
2018-08-10,250.0,250.0,250.0,2859.545748,117.783333,24.278017


In [149]:
# Compute feed stats for first week
feed_stats=feed.compute_feed_stats(trip_stats,dates=week)
feed_stats

,num_stops,num_routes,num_trips,num_trip_starts,num_trip_ends,service_distance,service_duration,service_speed,peak_num_trips,peak_start_time,peak_end_time,date
0,237,10,250,250,250,2859.545748,117.783333,24.278017,11,06:30:00,06:38:00,20180806
0,237,10,250,250,250,2859.545748,117.783333,24.278017,11,06:30:00,06:38:00,20180807
0,237,10,250,250,250,2859.545748,117.783333,24.278017,11,06:30:00,06:38:00,20180808
0,237,10,250,250,250,2859.545748,117.783333,24.278017,11,06:30:00,06:38:00,20180809
0,237,10,250,250,250,2859.545748,117.783333,24.278017,11,06:30:00,06:38:00,20180810
0,237,8,123,123,123,1606.106965,64.733333,24.811127,6,08:00:00,08:23:00,20180811
0,237,4,55,55,55,962.530167,33.800000,28.477224,4,08:00:00,08:07:00,20180812


In [150]:
# Compute route time series
rts = feed.compute_route_time_series(trip_stats, date, freq='12H')
rts

indicator           num_trip_ends              ... service_speed                      
route_id                    25492 25493 25494  ...         25499      25500      25516
datetime                                       ...                                    
2018-08-06 00:00:00          17.0   6.0  12.0  ...     11.893789  13.689405   0.000000
2018-08-06 12:00:00          19.0   8.0  14.0  ...      0.000000   0.000000  28.164349
2018-08-07 00:00:00           NaN   NaN   NaN  ...           NaN        NaN        NaN
2018-08-07 12:00:00           NaN   NaN   NaN  ...           NaN        NaN        NaN
2018-08-08 00:00:00           NaN   NaN   NaN  ...           NaN        NaN        NaN
2018-08-08 12:00:00           NaN   NaN   NaN  ...           NaN        NaN        NaN
2018-08-09 00:00:00           NaN   NaN   NaN  ...           NaN        NaN        NaN
2018-08-09 12:00:00           NaN   NaN   NaN  ...           NaN        NaN        NaN
2018-08-10 00:00:00          17.0   6.0  12.0  ...     11.893789  13.689405   0.000000
2018-08-10 12:00:00          19.0   8.0  14.0  ...      0.000000   0.000000  28.164349

[10 rows x 60 columns]

In [151]:
# Slice time series

inds = ['service_distance', 'service_duration', 'service_speed']
rids = ['3231','3232','3233','3234']

rts.loc[:, (inds, rids)]

datetime
2018-08-06 00:00:00
2018-08-06 12:00:00
2018-08-07 00:00:00
2018-08-07 12:00:00
2018-08-08 00:00:00
2018-08-08 12:00:00
2018-08-09 00:00:00
2018-08-09 12:00:00
2018-08-10 00:00:00
2018-08-10 12:00:00


In [152]:
# Compute trip locations for every hour

rng = pd.date_range('14/12/2018', periods=24, freq='H')
times = [t.strftime('%H:%M:%S') for t in rng]
loc = feed.locate_trips(date[0], times)
loc.head()

,trip_id,time,rel_dist,route_id,direction_id,shape_id,lon,lat
0,2660019,07:00:00,1.0,25492,0,82661 122507 164760,-76.50966,38.95293
1,2660020,08:00:00,1.0,25492,0,82661 122507 164760,-76.50966,38.95293
2,2660023,10:00:00,1.0,25492,0,82661 122507 164760,-76.50966,38.95293
3,2660024,11:00:00,1.0,25492,0,82661 122507 164760,-76.50966,38.95293
4,2660027,13:00:00,1.0,25492,0,82661 122507 164760,-76.50966,38.95293


In [153]:
feed.routes

,route_id,route_short_name,route_long_name,route_desc,route_url,route_color,route_text_color,route_type
0,25495,Green,Westfield Mall to Eastport,NaN,NaN,3B823F,ffffff,3
1,25500,NaN,State Shuttle,NaN,NaN,047AC8,FFFFFF,3
2,25516,Purple-N,Westfield Mall to Eastport,NaN,NaN,764D8E,ffffff,3
3,25496,Brown,Westfield Mall to Eastport,NaN,NaN,96613F,FFFFFF,3
4,25499,NaN,Circulator,NaN,NaN,047AC8,FFFFFF,3
5,25492,Orange,Downtown Annapolis to Forest Drive,NaN,NaN,EB8C2F,ffffff,3
6,25494,Yellow,Westfield Mall to Harbour Center,NaN,NaN,FDE520,000000,3
7,25493,Gold,Edgewater to Arnold/AACC,NaN,NaN,D8AB2E,FFFFFF,3
8,25497,Red,Westfield Mall to Eastport,NaN,NaN,D9392A,ffffff,3
9,25498,Purple-S,Westfield Mall to Eastport,NaN,NaN,764D8E,FFFFFF,3


In [154]:
feed.routes['route_id'].iat[0]

'25495'

In [126]:
# Build a route timetable
route_id = feed.routes['route_id'].iat[0]
feed.build_route_timetable(route_id, date).T

,1464,2338,1465,2339,1466,2340,1467,2341,1468,2342,1469,2343,1470,2344,1471,2345,1472,2346,1473,2347,1474,2348,1475,2349,1476,2350,1477,2351,1478,2352,1479,2353,1480,2354,1481,2355,1482,2356,1483,2357,...,2965,2066,2966,2067,2967,2068,2968,2069,2969,2070,2970,2071,2971,2072,2972,2073,2973,2074,2974,2075,2975,2076,2976,2077,2977,2078,2978,2079,2979,2080,2980,2081,2981,2082,2982,2083,2983,2084,2984,2985
trip_id,2660137,2660175,2660137,2660175,2660137,2660175,2660137,2660175,2660137,2660175,2660137,2660175,2660137,2660175,2660137,2660175,2660137,2660175,2660137,2660175,2660137,2660175,2660137,2660175,2660137,2660175,2660137,2660175,2660137,2660175,2660137,2660175,2660137,2660175,2660137,2660175,2660137,2660175,2660137,2660175,...,2660201,2660163,2660201,2660163,2660201,2660163,2660201,2660163,2660201,2660163,2660201,2660163,2660201,2660163,2660201,2660163,2660201,2660163,2660201,2660163,2660201,2660163,2660201,2660163,2660201,2660163,2660201,2660163,2660201,2660163,2660201,2660163,2660201,2660163,2660201,2660163,2660201,2660163,2660201,2660201
route_id,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,...,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495,25495
service_id,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,...,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333
trip_headsign,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,...,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Eastport,Westfield Mall,Westfield Mall
trip_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
direction_id,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,...,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1
shape_id,82666 122511 164764,82667 122512 164765,82666 122511 164764,82667 122512 164765,82666 122511 164764,82667 122512 164765,82666 122511 164764,82667 122512 164765,82666 122511 164764,82667 122512 164765,82666 122511 164764,82667 122512 164765,82666 122511 164764,82667 122512 164765,82666 122511 164764,82667 122512 164765,82666 122511 164764,82667 122512 164765,82666 122511 164764,82667 122512 164765,82666 122511 164764,82667 122512 164765,82666

In [156]:
feed.routes.route_id[feed.routes.route_id==route_id]

0    25495
Name: route_id, dtype: object

In [160]:
rids = feed.routes.route_id[feed.routes.route_id=='25495']
feed.map_routes(rids)

/usr/local/lib/python3.7/dist-packages/pandas/core/common.py:471: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  return func(obj, *args, **kwargs)


In [161]:
feed.trips.filter(['route_id','trip_id','shape_id']).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414
route_id,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25494,25494,25494,25494,...,25497,25497,25497,25497,25497,25497,25497,25497,25497,25497,25497,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25499,25499
trip_id,2660017,2660018,2660019,2660020,2660021,2660022,2660023,2660024,2660025,2660026,2660027,2660028,2660029,2660030,2660031,2660032,2660033,2660034,2660035,2660036,2660037,2660038,2660039,2660040,2660041,2660042,2660043,2660044,2660045,2660046,2660047,2660048,2660049,2660050,2660051,2660052,2660091,2660092,2660093,2660094,...,2660280,2660281,2660282,2660283,2660284,2660285,2660286,2660287,2660288,2660289,2660290,2660294,2660295,2660296,2660297,2660298,2660299,2660300,2660301,2660302,2660303,2660304,2660291,2660292,2660293,2660308,2660309,2660310,2660311,2660312,2660313,2660314,2660315,2660316,2660317,2660305,2660306,2660307,2660356,2660355
shape_id,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82662 122508 164761,82662 122508 164761,82662 122508 164761,82662 122508 164761,82662 122508 164761,82662 122508 164761,82662 122508 164761,82662 122508 164761,82662 122508 164761,82662 122508 164761,82662 122508 164761,82662 122508 164761,82662 122508 164761,82662 122508 164761,82662 122508 164761,82662 122508 164761,82662 122508 164761,82662 122508 164761,82665 122509 164762,82665 122509 164762,82665 122509 164762,82665 122509 164762,...,82673 122521 164774,82673 122521 164774,82673 122521 164774,82673 122521 164774,82673 122521 164774,82673 122521 164774,82673 122521 164774,82673 122521 164774,82673 122521 164774,82673 122521 164774,82673 122521 164774,82675 122522 164775,82675 122522 164775,82675 122522 164775,82675 122522 164775,82675 122522 164775,82675 122522 164775,82675 122522 164775,82675 122522 164775,82675 122522 164775,82675 122522 164775,82675 122522 164775,82675 122522 164775,82675 122522 164775,82675 122522 164775,82679 122523 164776,82679 122523 164776,82679 122523 164776,82679 122523 164776,82679 122523 164776,82679 122523 164776,82679 122523 164776,82679 122523 164776,82679 122523 164776,82679 122523 164776,82679 122523 164776,82679 122523 164776,82679 122523 164776,82680 122524 164777,82680 122524 164777


In [162]:
## FInd all the trips of specific route : slice rows
feed.trips.loc[lambda x : x.route_id == 'A']

,trip_id,route_id,service_id,trip_headsign,trip_short_name,direction_id,shape_id,wheelchair_accessible,bikes_allowed,block_id,block_name


In [163]:
## group all the trips by route 
feed.trips.groupby('route_id').agg(['mean']).T


,route_id,25492,25493,25494,25495,25496,25497,25498,25499,25500,25516
direction_id,mean,0.5,0.5,0.5,0.506494,0.5,0.506494,0.481481,0.0,0.0,0.518519
wheelchair_accessible,mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bikes_allowed,mean,1.0,1.0,1.0,1.000000,1.0,1.000000,1.000000,NaN,1.0,1.000000


In [164]:
## Comute the number of trips per route : split ,apply,combine
def my_agg(group):
  d={}
  d['num_trips']=group.shape[0]
  return pd.Series(d)


## Use method chaning 
(
    feed.trips
 # split 
 .groupby('route_id')
 #apply and combine
 .apply(my_agg)
 .reset_index()
 #Join in route short name from feeed routes
 .merge(feed.routes.filter(['route_id','route_short_name','route_long_name','route_type']))
 #sort values 
 .sort_values('num_trips',ascending=False).T
)

,4,3,5,2,1,0,6,9,7,8
route_id,25496,25495,25497,25494,25493,25492,25498,25516,25499,25500
num_trips,84,77,77,46,38,36,27,27,2,1
route_short_name,Brown,Green,Red,Yellow,Gold,Orange,Purple-S,Purple-N,NaN,NaN
route_long_name,Westfield Mall to Eastport,Westfield Mall to Eastport,Westfield Mall to Eastport,Westfield Mall to Harbour Center,Edgewater to Arnold/AACC,Downtown Annapolis to Forest Drive,Westfield Mall to Eastport,Westfield Mall to Eastport,Circulator,State Shuttle
route_type,3,3,3,3,3,3,3,3,3,3


In [165]:
(feed.trips
 # split 
 .groupby('route_id')
 #apply and combine
 .apply(my_agg))

,num_trips
route_id,
25492,36
25493,38
25494,46
25495,77
25496,84
25497,77
25498,27
25499,2
25500,1


In [166]:
feed.trips.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414
trip_id,2660017,2660018,2660019,2660020,2660021,2660022,2660023,2660024,2660025,2660026,2660027,2660028,2660029,2660030,2660031,2660032,2660033,2660034,2660035,2660036,2660037,2660038,2660039,2660040,2660041,2660042,2660043,2660044,2660045,2660046,2660047,2660048,2660049,2660050,2660051,2660052,2660091,2660092,2660093,2660094,...,2660280,2660281,2660282,2660283,2660284,2660285,2660286,2660287,2660288,2660289,2660290,2660294,2660295,2660296,2660297,2660298,2660299,2660300,2660301,2660302,2660303,2660304,2660291,2660292,2660293,2660308,2660309,2660310,2660311,2660312,2660313,2660314,2660315,2660316,2660317,2660305,2660306,2660307,2660356,2660355
route_id,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25492,25494,25494,25494,25494,...,25497,25497,25497,25497,25497,25497,25497,25497,25497,25497,25497,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25498,25499,25499
service_id,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,35333,...,35331,35331,35331,35331,35331,35331,35331,35331,35331,35331,35331,35332,35332,35332,35332,35332,35332,35332,35332,35332,35332,35332,35329,35329,35329,35332,35332,35332,35332,35332,35332,35332,35332,35332,35332,35329,35329,35329,35332,35329
trip_headsign,Forest Drive,Forest Drive,Forest Drive,Forest Drive,Forest Drive,Forest Drive,Forest Drive,Forest Drive,Forest Drive,Forest Drive,Forest Drive,Forest Drive,Forest Drive,Forest Drive,Forest Drive,Forest Drive,Forest Drive,Forest Drive,Downtown Annapolis,Downtown Annapolis,Downtown Annapolis,Downtown Annapolis,Downtown Annapolis,Downtown Annapolis,Downtown Annapolis,Downtown Annapolis,Downtown Annapolis,Downtown Annapolis,Downtown Annapolis,Downtown Annapolis,Downtown Annapolis,Downtown Annapolis,Downtown Annapolis,Downtown Annapolis,Downtown Annapolis,Downtown Annapolis,Harbour Center,Harbour Center,Harbour Center,Harbour Center,...,Westfield Mall,Westfield Mall,Westfield Mall,Westfield Mall,Westfield Mall,Westfield Mall,Westfield Mall,Westfield Mall,Westfield Mall,Westfield Mall,Westfield Mall,Eastport,Eastport,Eastport,Eastport,Eastport,Eastport,Eastport,Eastport,Eastport,Eastport,Eastport,Eastport,Eastport,Eastport,Westfield Mall,Westfield Mall,Westfield Mall,Westfield Mall,Westfield Mall,Westfield Mall,Westfield Mall,Westfield Mall,Westfield Mall,Westfield Mall,Westfield Mall,Westfield Mall,Westfield Mall,Loop,Loop
trip_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
direction_id,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0
shape_id,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82661 122507 164760,82662 122508 164761,82662 122508 164761,82662 122

In [172]:
# Compute screen line counts
DATE='20180904'

# Route stats
route_stats = feed.compute_route_stats(trip_stats, dates=[DATE])
route_stats.T

,0,1,2,3,4,5,6,7,8,9
route_id,25492,25493,25494,25495,25496,25497,25498,25499,25500,25516
route_short_name,Orange,Gold,Yellow,Green,Brown,Red,Purple-S,NaN,NaN,Purple-N
route_type,3,3,3,3,3,3,3,3,3,3
num_trips,36,14,26,54,52,54,6,1,1,6
num_trip_starts,36,14,26,54,52,54,6,1,1,6
num_trip_ends,36,14,26,54,52,54,6,1,1,6
is_loop,0,0,0,0,0,0,0,1,1,0
is_bidirectional,1,1,1,1,1,1,1,0,0,1
start_time,05:30:00,06:00:00,06:00:00,05:30:00,05:45:00,05:30:00,18:57:00,07:30:00,06:30:00,19:00:00
end_time,18:52:00,19:56:00,18:52:00,18:54:00,19:08:00,18:55:00,22:39:00,07:50:00,06:50:00,22:36:00


In [173]:
feed.get_routes(DATE)

,route_id,route_short_name,route_long_name,route_desc,route_url,route_color,route_text_color,route_type
0,25495,Green,Westfield Mall to Eastport,NaN,NaN,3B823F,ffffff,3
1,25500,NaN,State Shuttle,NaN,NaN,047AC8,FFFFFF,3
2,25516,Purple-N,Westfield Mall to Eastport,NaN,NaN,764D8E,ffffff,3
3,25496,Brown,Westfield Mall to Eastport,NaN,NaN,96613F,FFFFFF,3
4,25499,NaN,Circulator,NaN,NaN,047AC8,FFFFFF,3
5,25492,Orange,Downtown Annapolis to Forest Drive,NaN,NaN,EB8C2F,ffffff,3
6,25494,Yellow,Westfield Mall to Harbour Center,NaN,NaN,FDE520,000000,3
7,25493,Gold,Edgewater to Arnold/AACC,NaN,NaN,D8AB2E,FFFFFF,3
8,25497,Red,Westfield Mall to Eastport,NaN,NaN,D9392A,ffffff,3
9,25498,Purple-S,Westfield Mall to Eastport,NaN,NaN,764D8E,FFFFFF,3


In [174]:
# Join in route long name
route_stats = route_stats.merge(feed.routes.filter(["route_id", "route_long_name"]))

cols = [
    "route_id",
    "route_short_name", 
    "route_long_name", 
    "route_type", 
    "start_time",
    "end_time",
    "num_trips",
    "max_headway",
    "mean_trip_distance",
    "mean_trip_duration",
    "service_speed",
]

In [175]:
# The most/least frequent routes are
(
    route_stats
    .filter(cols)
    .sort_values("max_headway")
    .T
)

,3,4,5,0,2,1,6,7,8,9
route_id,25495,25496,25497,25492,25494,25493,25498,25499,25500,25516
route_short_name,Green,Brown,Red,Orange,Yellow,Gold,Purple-S,NaN,NaN,Purple-N
route_long_name,Westfield Mall to Eastport,Westfield Mall to Eastport,Westfield Mall to Eastport,Downtown Annapolis to Forest Drive,Westfield Mall to Harbour Center,Edgewater to Arnold/AACC,Westfield Mall to Eastport,Circulator,State Shuttle,Westfield Mall to Eastport
route_type,3,3,3,3,3,3,3,3,3,3
start_time,05:30:00,05:45:00,05:30:00,05:30:00,06:00:00,06:00:00,18:57:00,07:30:00,06:30:00,19:00:00
end_time,18:54:00,19:08:00,18:55:00,18:52:00,18:52:00,19:56:00,22:39:00,07:50:00,06:50:00,22:36:00
num_trips,54,52,54,36,26,14,6,1,1,6
max_headway,30,30,30,45,60,120,NaN,NaN,NaN,NaN
mean_trip_distance,8.07366,15.4429,9.81291,7.61634,9.19777,28.1192,14.6335,3.9646,4.56313,14.5516
mean_trip_duration,0.391667,0.616667,0.408333,0.308333,0.433333,0.941667,0.533333,0.333333,0.333333,0.516667


In [176]:
# The shortest/longest routes are
(
    route_stats
    .filter(cols)
    .sort_values("mean_trip_distance")
    .T
)

,7,8,0,3,2,5,9,6,4,1
route_id,25499,25500,25492,25495,25494,25497,25516,25498,25496,25493
route_short_name,NaN,NaN,Orange,Green,Yellow,Red,Purple-N,Purple-S,Brown,Gold
route_long_name,Circulator,State Shuttle,Downtown Annapolis to Forest Drive,Westfield Mall to Eastport,Westfield Mall to Harbour Center,Westfield Mall to Eastport,Westfield Mall to Eastport,Westfield Mall to Eastport,Westfield Mall to Eastport,Edgewater to Arnold/AACC
route_type,3,3,3,3,3,3,3,3,3,3
start_time,07:30:00,06:30:00,05:30:00,05:30:00,06:00:00,05:30:00,19:00:00,18:57:00,05:45:00,06:00:00
end_time,07:50:00,06:50:00,18:52:00,18:54:00,18:52:00,18:55:00,22:36:00,22:39:00,19:08:00,19:56:00
num_trips,1,1,36,54,26,54,6,6,52,14
max_headway,NaN,NaN,45,30,60,30,NaN,NaN,30,120
mean_trip_distance,3.9646,4.56313,7.61634,8.07366,9.19777,9.81291,14.5516,14.6335,15.4429,28.1192
mean_trip_duration,0.333333,0.333333,0.308333,0.391667,0.433333,0.408333,0.516667,0.533333,0.616667,0.941667


In [177]:
# The slowest/fastest routes are
(
    route_stats
    .filter(cols)
    .sort_values("service_speed")
    .T
)

,7,8,3,2,5,0,4,6,9,1
route_id,25499,25500,25495,25494,25497,25492,25496,25498,25516,25493
route_short_name,NaN,NaN,Green,Yellow,Red,Orange,Brown,Purple-S,Purple-N,Gold
route_long_name,Circulator,State Shuttle,Westfield Mall to Eastport,Westfield Mall to Harbour Center,Westfield Mall to Eastport,Downtown Annapolis to Forest Drive,Westfield Mall to Eastport,Westfield Mall to Eastport,Westfield Mall to Eastport,Edgewater to Arnold/AACC
route_type,3,3,3,3,3,3,3,3,3,3
start_time,07:30:00,06:30:00,05:30:00,06:00:00,05:30:00,05:30:00,05:45:00,18:57:00,19:00:00,06:00:00
end_time,07:50:00,06:50:00,18:54:00,18:52:00,18:55:00,18:52:00,19:08:00,22:39:00,22:36:00,19:56:00
num_trips,1,1,54,26,54,36,52,6,6,14
max_headway,NaN,NaN,30,60,30,45,30,NaN,NaN,120
mean_trip_distance,3.9646,4.56313,8.07366,9.19777,9.81291,7.61634,15.4429,14.6335,14.5516,28.1192
mean_trip_duration,0.333333,0.333333,0.391667,0.433333,0.408333,0.308333,0.616667,0.533333,0.516667,0.941667


In [180]:
# Map routes

# Map our extreme routes
feed.map_routes(route_ids=['25499','25500','25495'])


/usr/local/lib/python3.7/dist-packages/pandas/core/common.py:471: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  return func(obj, *args, **kwargs)


In [183]:
feed=gk.read_feed('/content/gtfs (4).zip',dist_units='km')

In [185]:
feed=feed.append_dist_to_stop_times()

In [187]:
feed.stop_times.head().T

,0,1,2,3,4
trip_id,2659985,2659985,2659985,2659985,2659985
arrival_time,07:15:00,NaN,NaN,NaN,07:21:00
departure_time,07:15:00,NaN,NaN,NaN,07:21:00
stop_id,1025862,1025881,1025921,1025764,1025765
stop_sequence,0,1,2,3,4
timepoint,1,1,1,1,1
shape_dist_traveled,NaN,NaN,NaN,NaN,1.98464


In [189]:
first_week=feed.get_stops()

In [192]:
trip_stats=feed.compute_trip_stats()
trip_stats

,trip_id,route_id,route_short_name,route_type,direction_id,shape_id,num_stops,start_time,end_time,start_stop_id,end_stop_id,is_loop,duration,distance,speed
32,2660017,25492,Orange,3,0,82661 122507 164760,22,05:30:00,05:51:00,1025801,1025798,0,0.350000,5.990242,17.114978
33,2660018,25492,Orange,3,0,82661 122507 164760,22,06:15:00,06:36:00,1025801,1025798,0,0.350000,9.260439,26.458398
34,2660019,25492,Orange,3,0,82661 122507 164760,22,07:00:00,07:21:00,1025801,1025798,0,0.350000,9.260439,26.458398
35,2660020,25492,Orange,3,0,82661 122507 164760,22,07:45:00,08:06:00,1025801,1025798,0,0.350000,9.260439,26.458398
36,2660021,25492,Orange,3,0,82661 122507 164760,22,08:30:00,08:51:00,1025801,1025798,0,0.350000,9.260439,26.458398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,2660353,25516,Purple-N,3,1,85051 122515 164768,35,18:20:00,18:51:00,1025756,1025862,0,0.516667,16.180920,31.317910
346,2660331,25516,Purple-N,3,1,85051 122515 164768,35,19:35:00,20:06:00,1025756,1025862,0,0.516667,0.000000,0.000000
359,2660354,25516,Purple-N,3,1,85051 122515 164768,35,19:35:00,20:06:00,1025756,1025862,0,0.516667,16.180920,31.317910
347,2660332,25516,Purple-N,3,1,85051 122515 164768,35,20:50:00,21:21:00,1025756,1025862,0,0.516667,16.180920,31.317910


In [190]:
feed.compute_feed_stats()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,1025801,NaN,West St. and Calvert St.,NaN,38.978286,-76.496264,NaN,0,NaN,NaN,NaN
1,1025874,NaN,West St. at Loews Hotel,NaN,38.977833,-76.498248,NaN,0,NaN,NaN,NaN
2,1025805,NaN,West St. and Madison Pl.,NaN,38.977157,-76.500693,NaN,0,NaN,NaN,NaN
3,1025810,NaN,West St. and Taylor Ave.,NaN,38.976390,-76.504420,NaN,0,NaN,NaN,NaN
4,1025794,NaN,Spa Rd. and Carraway Ln.,NaN,38.975375,-76.507002,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
232,1025831,NaN,Bestgate Rd. at Commerce Park Dr.,NaN,38.993041,-76.542844,NaN,0,NaN,NaN,NaN
233,1025951,NaN,Tyler Ave. and Woods Dr.,NaN,38.959868,-76.498293,NaN,0,NaN,NaN,NaN
234,1025950,NaN,Tyler Ave. and Janwall St.,NaN,38.957592,-76.502375,NaN,0,NaN,NaN,NaN
235,1025797,NaN,Tyler Ave. and President St.,NaN,38.960483,-76.494107,NaN,0,NaN,NaN,NaN
